### LDA Probabilistic Modeling
- LDA + CountVectorize (3 topics)
- LDA + CountVectorize (4 topics)
- Don't use TF-IDF with LDA

In [1]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.util import ngrams
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import re

from gensim import corpora, models, similarities, matutils
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jennihawk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#got tweets at 1:15 Pacific Time 10/14/22
tweets = pd.read_csv('/Users/jennihawk/Documents/Data Science Projects/Project_NLP/TweetBatch3.csv')
tweets

,text,cleaned
0,@ReallyAmerican1 #Roevember and\n#ForThePeople...,roevember and forthepeople and votebluein2022...
1,RT @sandibachom: IS THIS THING ON???!!This is ...,rt is this thing on this is pathetic acting se...
2,RT @sandibachom: IS THIS THING ON???!!This is ...,rt is this thing on this is pathetic acting se...
3,RT @tleehumphrey: Today is the beginning of th...,rt today is the beginning of the inquiry into ...
4,RT @AdamKinzinger: Mitch McConnell.\nKevin McC...,rt mitch mcconnell kevin mccarthy they both kn...
...,...,...
34988,RT @Adrian_Fontes: The January 6th committee j...,rt fontes the january 6th committee just concl...
34989,#January6thCommitteeHearings and everyone runn...,january6thcommitteehearings and everyone runn...
34990,RT @sandibachom: IS THIS THING ON???!!This is ...,rt is this thing on this is pathetic acting se...
34991,So they are gonna subpoena Trump I am guessing...,so they are gonna subpoena trump am guessing t...


### Remove stop words from cleaned column

In [3]:
stop_words = stopwords.words('english')
stop_words.extend(['rt', 'january', 'january6thcommitteehearings', 'ja'])
tweets['cleaned'] = tweets['cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [4]:
#tweets.head()

### Tokenize Words in Cleaned Column

In [5]:
tweets['tokenized'] = tweets.apply(lambda row: nltk.word_tokenize(row['cleaned']), axis=1)

In [6]:
tweets.head()

,text,cleaned,tokenized
0,@ReallyAmerican1 #Roevember and\n#ForThePeople...,roevember forthepeople votebluein2022 standwit...,"[roevember, forthepeople, votebluein2022, stan..."
1,RT @sandibachom: IS THIS THING ON???!!This is ...,thing pathetic acting sec defense chris miller...,"[thing, pathetic, acting, sec, defense, chris,..."
2,RT @sandibachom: IS THIS THING ON???!!This is ...,thing pathetic acting sec defense chris miller...,"[thing, pathetic, acting, sec, defense, chris,..."
3,RT @tleehumphrey: Today is the beginning of th...,today beginning inquiry trudeau gov use emerge...,"[today, beginning, inquiry, trudeau, gov, use,..."
4,RT @AdamKinzinger: Mitch McConnell.\nKevin McC...,mitch mcconnell kevin mccarthy knew trump resp...,"[mitch, mcconnell, kevin, mccarthy, knew, trum..."


## Countvectorizer + LDA 

### Countvectorizer
- a method to convert text to numerical data
- By default, Countvectorizer converts the text to lowercase and uses word-level tokenization. 
- Countervectorize will do word tokenization not other types of tokenization. CountVectorizer takes list. 

In [7]:
corpus = list(tweets.cleaned)

In [8]:
#turn corpus into a document-term matrix
vectorizer = CountVectorizer()
doc_term = vectorizer.fit_transform(corpus)
doc_term_df = pd.DataFrame(doc_term.toarray(), columns=vectorizer.get_feature_names())

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [9]:
doc_term_df.shape

(34993, 9400)

In [10]:
#doc_term_df.iloc[:, : 100]

### LDA: Reduce Dimensionality 
- Probabilistic Modeling using gensim
- Increase the number of passes to get more stable results.

#### Fit LDA Model - 4 Topics

In [11]:
#Fit an LDA model using `LdaModel`. `passes` = number of times corpus scanned. Save the fitted model as `lda`.
term_doc = doc_term.transpose()
corpus = matutils.Sparse2Corpus(term_doc)
id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())
lda = models.LdaModel(corpus=corpus, num_topics=4, id2word=id2word, random_state=100, passes=10)

lda

2023-07-17 11:06:39,984 : INFO : using symmetric alpha at 0.25
2023-07-17 11:06:39,985 : INFO : using symmetric eta at 0.25
2023-07-17 11:06:39,987 : INFO : using serial LDA version on this node
2023-07-17 11:06:39,990 : INFO : running online (multi-pass) LDA training, 4 topics, 10 passes over the supplied corpus of 34993 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2023-07-17 11:06:40,046 : INFO : PROGRESS: pass 0, at document #2000/34993
2023-07-17 11:06:40,761 : INFO : merging changes from 2000 documents into a model of 34993 documents
2023-07-17 11:06:40,763 : INFO : topic #0 (0.250): 0.043*"knew" + 0.029*"pelosi" + 0.026*"trump" + 0.025*"would" + 0.025*"nancy" + 0.022*"th" + 0.022*"protests" + 0.022*"film" + 0.022*"advance" + 0.022*"daughter"
2023-07-17 11:06:40,764 : INFO : topic #1 (0.250): 0.081*"trump" + 0.050*"knew" + 0.048*"mitch" + 0.048*"responsible" + 0.048*"backed"

2023-07-17 11:06:42,218 : INFO : topic diff=0.361766, rho=0.377964
2023-07-17 11:06:42,224 : INFO : PROGRESS: pass 0, at document #16000/34993
2023-07-17 11:06:42,406 : INFO : merging changes from 2000 documents into a model of 34993 documents
2023-07-17 11:06:42,408 : INFO : topic #0 (0.250): 0.030*"knew" + 0.016*"amp" + 0.013*"pelosi" + 0.012*"trump" + 0.012*"today" + 0.010*"like" + 0.009*"day" + 0.009*"going" + 0.008*"still" + 0.008*"nancy"
2023-07-17 11:06:42,409 : INFO : topic #1 (0.250): 0.093*"trump" + 0.086*"knew" + 0.070*"called" + 0.069*"responsible" + 0.069*"mcconnell" + 0.069*"mccarthy" + 0.069*"kevin" + 0.069*"mitch" + 0.068*"backed" + 0.013*"voted"
2023-07-17 11:06:42,409 : INFO : topic #2 (0.250): 0.039*"maga" + 0.036*"democracy" + 0.036*"country" + 0.035*"gop" + 0.033*"violence" + 0.032*"without" + 0.031*"love" + 0.031*"january6thcomm" + 0.030*"mark" + 0.030*"overthrowing"
2023-07-17 11:06:42,410 : INFO : topic #3 (0.250): 0.096*"trump" + 0.018*"decided" + 0.018*"lost" 

2023-07-17 11:06:44,055 : INFO : topic #1 (0.250): 0.098*"trump" + 0.083*"knew" + 0.063*"responsible" + 0.063*"backed" + 0.063*"mcconnell" + 0.062*"called" + 0.062*"mitch" + 0.062*"mccarthy" + 0.062*"kevin" + 0.023*"voted"
2023-07-17 11:06:44,055 : INFO : topic #2 (0.250): 0.031*"trump" + 0.021*"maga" + 0.020*"thing" + 0.019*"chris" + 0.019*"country" + 0.018*"national" + 0.018*"gop" + 0.018*"miller" + 0.018*"person" + 0.018*"acting"
2023-07-17 11:06:44,056 : INFO : topic #3 (0.250): 0.102*"trump" + 0.022*"lost" + 0.021*"decided" + 0.020*"video" + 0.018*"coup" + 0.018*"help" + 0.018*"new" + 0.017*"roger" + 0.017*"stone" + 0.016*"yet"
2023-07-17 11:06:44,056 : INFO : topic diff=0.241935, rho=0.258199
2023-07-17 11:06:44,062 : INFO : PROGRESS: pass 0, at document #32000/34993
2023-07-17 11:06:44,230 : INFO : merging changes from 2000 documents into a model of 34993 documents
2023-07-17 11:06:44,232 : INFO : topic #0 (0.250): 0.030*"knew" + 0.013*"amp" + 0.011*"pelosi" + 0.010*"trump" + 0.

2023-07-17 11:06:45,389 : INFO : topic #3 (0.250): 0.099*"trump" + 0.018*"decided" + 0.017*"video" + 0.017*"pelosi" + 0.016*"6th" + 0.015*"lost" + 0.014*"attack" + 0.013*"help" + 0.013*"coup" + 0.012*"stone"
2023-07-17 11:06:45,389 : INFO : topic diff=0.235611, rho=0.226476
2023-07-17 11:06:45,395 : INFO : PROGRESS: pass 1, at document #10000/34993
2023-07-17 11:06:45,551 : INFO : merging changes from 2000 documents into a model of 34993 documents
2023-07-17 11:06:45,553 : INFO : topic #0 (0.250): 0.031*"knew" + 0.014*"pelosi" + 0.014*"amp" + 0.011*"would" + 0.010*"trump" + 0.010*"like" + 0.009*"nancy" + 0.009*"still" + 0.007*"fbi" + 0.007*"held"
2023-07-17 11:06:45,553 : INFO : topic #1 (0.250): 0.086*"trump" + 0.075*"knew" + 0.064*"responsible" + 0.064*"called" + 0.063*"mcconnell" + 0.063*"mccarthy" + 0.063*"mitch" + 0.063*"kevin" + 0.063*"backed" + 0.024*"chair"
2023-07-17 11:06:45,553 : INFO : topic #2 (0.250): 0.039*"maga" + 0.035*"country" + 0.035*"democracy" + 0.034*"without" + 

2023-07-17 11:06:47,117 : INFO : merging changes from 2000 documents into a model of 34993 documents
2023-07-17 11:06:47,119 : INFO : topic #0 (0.250): 0.026*"knew" + 0.014*"pelosi" + 0.013*"amp" + 0.010*"trump" + 0.009*"like" + 0.009*"today" + 0.007*"going" + 0.007*"still" + 0.007*"jan" + 0.007*"held"
2023-07-17 11:06:47,119 : INFO : topic #1 (0.250): 0.098*"trump" + 0.091*"knew" + 0.068*"called" + 0.068*"mcconnell" + 0.068*"responsible" + 0.067*"mccarthy" + 0.067*"mitch" + 0.067*"kevin" + 0.067*"backed" + 0.019*"subpoena"
2023-07-17 11:06:47,120 : INFO : topic #2 (0.250): 0.031*"maga" + 0.027*"country" + 0.026*"democracy" + 0.026*"trump" + 0.025*"violence" + 0.023*"love" + 0.023*"without" + 0.023*"gop" + 0.023*"january6thcomm" + 0.022*"mark"
2023-07-17 11:06:47,120 : INFO : topic #3 (0.250): 0.106*"trump" + 0.024*"decided" + 0.021*"lost" + 0.018*"video" + 0.017*"help" + 0.017*"new" + 0.017*"coup" + 0.017*"roger" + 0.017*"stone" + 0.015*"yet"
2023-07-17 11:06:47,120 : INFO : topic dif

...

2023-07-17 11:07:10,189 : INFO : topic #2 (0.250): 0.037*"maga" + 0.034*"country" + 0.033*"democracy" + 0.033*"love" + 0.032*"without" + 0.032*"violence" + 0.030*"gop" + 0.030*"january6thcomm" + 0.029*"mark" + 0.029*"overthrowing"
2023-07-17 11:07:10,189 : INFO : topic #3 (0.250): 0.109*"trump" + 0.022*"decided" + 0.020*"video" + 0.018*"6th" + 0.018*"lost" + 0.017*"attack" + 0.016*"help" + 0.016*"pelosi" + 0.015*"coup" + 0.015*"stone"
2023-07-17 11:07:10,189 : INFO : topic diff=0.160972, rho=0.194270
2023-07-17 11:07:10,196 : INFO : PROGRESS: pass 8, at document #10000/34993
2023-07-17 11:07:10,341 : INFO : merging changes from 2000 documents into a model of 34993 documents
2023-07-17 11:07:10,342 : INFO : topic #0 (0.250): 0.025*"knew" + 0.015*"trump" + 0.014*"pelosi" + 0.010*"amp" + 0.009*"nancy" + 0.009*"would" + 0.008*"like" + 0.008*"still" + 0.006*"held" + 0.006*"fbi"
2023-07-17 11:07:10,343 : INFO : topic #1 (0.250): 0.087*"trump" + 0.078*"knew" + 0.065*"responsible" + 0.065*"cal

2023-07-17 11:07:11,622 : INFO : topic diff=0.142106, rho=0.194270
2023-07-17 11:07:11,628 : INFO : PROGRESS: pass 8, at document #24000/34993
2023-07-17 11:07:11,785 : INFO : merging changes from 2000 documents into a model of 34993 documents
2023-07-17 11:07:11,786 : INFO : topic #0 (0.250): 0.023*"knew" + 0.017*"trump" + 0.014*"pelosi" + 0.011*"amp" + 0.008*"like" + 0.008*"today" + 0.006*"nancy" + 0.006*"going" + 0.006*"held" + 0.006*"still"
2023-07-17 11:07:11,787 : INFO : topic #1 (0.250): 0.098*"trump" + 0.092*"knew" + 0.069*"called" + 0.068*"mcconnell" + 0.068*"responsible" + 0.068*"mccarthy" + 0.068*"mitch" + 0.068*"kevin" + 0.068*"backed" + 0.020*"subpoena"
2023-07-17 11:07:11,787 : INFO : topic #2 (0.250): 0.032*"maga" + 0.028*"country" + 0.027*"democracy" + 0.026*"violence" + 0.025*"trump" + 0.025*"love" + 0.025*"without" + 0.024*"gop" + 0.024*"january6thcomm" + 0.023*"mark"
2023-07-17 11:07:11,788 : INFO : topic #3 (0.250): 0.109*"trump" + 0.026*"decided" + 0.023*"lost" + 0

2023-07-17 11:07:13,027 : INFO : topic #1 (0.250): 0.096*"trump" + 0.083*"knew" + 0.065*"responsible" + 0.064*"mcconnell" + 0.064*"called" + 0.064*"mitch" + 0.064*"backed" + 0.064*"mccarthy" + 0.064*"kevin" + 0.022*"subpoena"
2023-07-17 11:07:13,027 : INFO : topic #2 (0.250): 0.030*"trump" + 0.026*"maga" + 0.022*"country" + 0.021*"love" + 0.021*"democracy" + 0.020*"violence" + 0.020*"without" + 0.019*"thing" + 0.019*"january6thcomm" + 0.019*"gop"
2023-07-17 11:07:13,027 : INFO : topic #3 (0.250): 0.111*"trump" + 0.022*"decided" + 0.021*"lost" + 0.019*"attack" + 0.019*"video" + 0.018*"help" + 0.018*"coup" + 0.017*"stone" + 0.017*"new" + 0.017*"roger"
2023-07-17 11:07:13,028 : INFO : topic diff=0.164985, rho=0.190705
2023-07-17 11:07:13,033 : INFO : PROGRESS: pass 9, at document #4000/34993
2023-07-17 11:07:13,187 : INFO : merging changes from 2000 documents into a model of 34993 documents
2023-07-17 11:07:13,189 : INFO : topic #0 (0.250): 0.027*"knew" + 0.016*"pelosi" + 0.015*"trump" + 

2023-07-17 11:07:14,179 : INFO : topic #3 (0.250): 0.108*"trump" + 0.022*"decided" + 0.021*"lost" + 0.020*"video" + 0.017*"6th" + 0.017*"help" + 0.016*"stone" + 0.016*"coup" + 0.016*"roger" + 0.016*"attack"
2023-07-17 11:07:14,179 : INFO : topic diff=0.127641, rho=0.190705
2023-07-17 11:07:14,185 : INFO : PROGRESS: pass 9, at document #18000/34993
2023-07-17 11:07:14,341 : INFO : merging changes from 2000 documents into a model of 34993 documents
2023-07-17 11:07:14,343 : INFO : topic #0 (0.250): 0.023*"knew" + 0.015*"trump" + 0.013*"pelosi" + 0.012*"amp" + 0.008*"nancy" + 0.008*"like" + 0.008*"held" + 0.007*"today" + 0.007*"still" + 0.007*"would"
2023-07-17 11:07:14,343 : INFO : topic #1 (0.250): 0.094*"trump" + 0.089*"knew" + 0.070*"called" + 0.069*"responsible" + 0.068*"mcconnell" + 0.068*"mccarthy" + 0.068*"mitch" + 0.068*"kevin" + 0.068*"backed" + 0.017*"subpoena"
2023-07-17 11:07:14,344 : INFO : topic #2 (0.250): 0.038*"maga" + 0.034*"country" + 0.033*"democracy" + 0.031*"violenc

2023-07-17 11:07:15,841 : INFO : merging changes from 2000 documents into a model of 34993 documents
2023-07-17 11:07:15,843 : INFO : topic #0 (0.250): 0.024*"knew" + 0.015*"trump" + 0.011*"pelosi" + 0.011*"amp" + 0.008*"like" + 0.007*"today" + 0.007*"still" + 0.006*"going" + 0.005*"fbi" + 0.005*"jan"
2023-07-17 11:07:15,843 : INFO : topic #1 (0.250): 0.100*"trump" + 0.089*"knew" + 0.065*"responsible" + 0.065*"called" + 0.065*"mcconnell" + 0.065*"backed" + 0.065*"mitch" + 0.065*"mccarthy" + 0.064*"kevin" + 0.023*"subpoena"
2023-07-17 11:07:15,844 : INFO : topic #2 (0.250): 0.033*"trump" + 0.023*"maga" + 0.021*"thing" + 0.021*"chris" + 0.020*"country" + 0.020*"national" + 0.019*"democracy" + 0.019*"miller" + 0.019*"person" + 0.019*"acting"
2023-07-17 11:07:15,844 : INFO : topic #3 (0.250): 0.113*"trump" + 0.025*"lost" + 0.024*"decided" + 0.021*"video" + 0.020*"help" + 0.020*"coup" + 0.020*"new" + 0.019*"stone" + 0.019*"roger" + 0.018*"attack"
2023-07-17 11:07:15,844 : INFO : topic diff=

### Topic Optimizations to consider when results look fuzzy:
- Increase the number of passes to get more stable results.
- Change the number of topics
- Clean up the text more in the CountVectorizer step: adding to the stop word list, removing common words, etc.

In [12]:
lda.print_topics()

2023-07-17 11:07:16,261 : INFO : topic #0 (0.250): 0.027*"knew" + 0.015*"trump" + 0.011*"pelosi" + 0.010*"amp" + 0.009*"today" + 0.008*"like" + 0.008*"still" + 0.006*"fbi" + 0.006*"going" + 0.006*"security"
2023-07-17 11:07:16,262 : INFO : topic #1 (0.250): 0.099*"trump" + 0.083*"knew" + 0.064*"responsible" + 0.064*"mcconnell" + 0.063*"called" + 0.063*"mitch" + 0.063*"backed" + 0.063*"mccarthy" + 0.063*"kevin" + 0.025*"subpoena"
2023-07-17 11:07:16,262 : INFO : topic #2 (0.250): 0.034*"trump" + 0.022*"maga" + 0.022*"chris" + 0.022*"thing" + 0.021*"national" + 0.020*"miller" + 0.020*"person" + 0.020*"sec" + 0.020*"acting" + 0.019*"defense"
2023-07-17 11:07:16,263 : INFO : topic #3 (0.250): 0.112*"trump" + 0.022*"lost" + 0.021*"decided" + 0.020*"attack" + 0.018*"coup" + 0.018*"help" + 0.018*"video" + 0.017*"new" + 0.017*"stone" + 0.017*"roger"


[(0,
  '0.027*"knew" + 0.015*"trump" + 0.011*"pelosi" + 0.010*"amp" + 0.009*"today" + 0.008*"like" + 0.008*"still" + 0.006*"fbi" + 0.006*"going" + 0.006*"security"'),
 (1,
  '0.099*"trump" + 0.083*"knew" + 0.064*"responsible" + 0.064*"mcconnell" + 0.063*"called" + 0.063*"mitch" + 0.063*"backed" + 0.063*"mccarthy" + 0.063*"kevin" + 0.025*"subpoena"'),
 (2,
  '0.034*"trump" + 0.022*"maga" + 0.022*"chris" + 0.022*"thing" + 0.021*"national" + 0.020*"miller" + 0.020*"person" + 0.020*"sec" + 0.020*"acting" + 0.019*"defense"'),
 (3,
  '0.112*"trump" + 0.022*"lost" + 0.021*"decided" + 0.020*"attack" + 0.018*"coup" + 0.018*"help" + 0.018*"video" + 0.017*"new" + 0.017*"stone" + 0.017*"roger"')]

#### Fit LDA Model - 3 Topics

In [13]:
term_doc = doc_term.transpose()
corpus = matutils.Sparse2Corpus(term_doc)
id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())
lda2 = models.LdaModel(corpus=corpus, num_topics=3, id2word=id2word, passes=10)
### END SOLUTION
lda2

2023-07-17 11:07:16,268 : INFO : using symmetric alpha at 0.3333333333333333
2023-07-17 11:07:16,269 : INFO : using symmetric eta at 0.3333333333333333
2023-07-17 11:07:16,270 : INFO : using serial LDA version on this node
2023-07-17 11:07:16,272 : INFO : running online (multi-pass) LDA training, 3 topics, 10 passes over the supplied corpus of 34993 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2023-07-17 11:07:16,278 : INFO : PROGRESS: pass 0, at document #2000/34993
2023-07-17 11:07:17,024 : INFO : merging changes from 2000 documents into a model of 34993 documents
2023-07-17 11:07:17,026 : INFO : topic #0 (0.333): 0.042*"trump" + 0.028*"pelosi" + 0.025*"knew" + 0.019*"nancy" + 0.011*"would" + 0.011*"th" + 0.011*"daughter" + 0.011*"democracy" + 0.010*"hoax" + 0.010*"protests"
2023-07-17 11:07:17,027 : INFO : topic #1 (0.333): 0.029*"trump" + 0.025*"mark" + 0.024*"democracy" + 0.

2023-07-17 11:07:18,679 : INFO : topic #1 (0.333): 0.032*"trump" + 0.028*"democracy" + 0.027*"mark" + 0.019*"maga" + 0.018*"country" + 0.018*"gop" + 0.017*"violence" + 0.016*"without" + 0.016*"love" + 0.015*"donald"
2023-07-17 11:07:18,680 : INFO : topic #2 (0.333): 0.133*"trump" + 0.102*"knew" + 0.071*"called" + 0.069*"responsible" + 0.069*"mcconnell" + 0.069*"mccarthy" + 0.069*"mitch" + 0.069*"kevin" + 0.069*"backed" + 0.011*"6th"
2023-07-17 11:07:18,680 : INFO : topic diff=0.328113, rho=0.333333
2023-07-17 11:07:18,998 : INFO : -6.200 per-word bound, 73.5 perplexity estimate based on a held-out corpus of 2000 documents with 20829 words
2023-07-17 11:07:18,999 : INFO : PROGRESS: pass 0, at document #20000/34993
2023-07-17 11:07:19,174 : INFO : merging changes from 2000 documents into a model of 34993 documents
2023-07-17 11:07:19,175 : INFO : topic #0 (0.333): 0.044*"trump" + 0.016*"pelosi" + 0.013*"election" + 0.011*"trumpcoupattempt" + 0.010*"people" + 0.009*"democracy" + 0.009*"am

2023-07-17 11:07:20,689 : INFO : topic #1 (0.333): 0.041*"trump" + 0.025*"democracy" + 0.021*"donald" + 0.020*"subpoena" + 0.020*"mark" + 0.019*"voted" + 0.016*"coup" + 0.016*"history" + 0.015*"testify" + 0.015*"unanimously"
2023-07-17 11:07:20,690 : INFO : topic #2 (0.333): 0.141*"trump" + 0.108*"knew" + 0.061*"responsible" + 0.060*"mcconnell" + 0.060*"called" + 0.060*"mitch" + 0.060*"backed" + 0.060*"mccarthy" + 0.060*"kevin" + 0.017*"6th"
2023-07-17 11:07:20,690 : INFO : topic diff=0.177129, rho=0.235702
2023-07-17 11:07:20,695 : INFO : PROGRESS: pass 1, at document #2000/34993
2023-07-17 11:07:20,875 : INFO : merging changes from 2000 documents into a model of 34993 documents
2023-07-17 11:07:20,876 : INFO : topic #0 (0.333): 0.039*"trump" + 0.018*"pelosi" + 0.010*"get" + 0.008*"nancy" + 0.008*"election" + 0.008*"hours" + 0.007*"come" + 0.007*"people" + 0.007*"happened" + 0.007*"hey"
2023-07-17 11:07:20,877 : INFO : topic #1 (0.333): 0.038*"trump" + 0.026*"democracy" + 0.022*"mark"

2023-07-17 11:07:22,249 : INFO : topic #1 (0.333): 0.032*"trump" + 0.029*"democracy" + 0.027*"mark" + 0.019*"maga" + 0.019*"country" + 0.019*"gop" + 0.018*"violence" + 0.017*"without" + 0.017*"love" + 0.016*"january6thcomm"
2023-07-17 11:07:22,249 : INFO : topic #2 (0.333): 0.137*"trump" + 0.114*"knew" + 0.072*"called" + 0.070*"responsible" + 0.070*"mcconnell" + 0.070*"mccarthy" + 0.070*"mitch" + 0.070*"kevin" + 0.070*"backed" + 0.013*"6th"
2023-07-17 11:07:22,249 : INFO : topic diff=0.203592, rho=0.226476
2023-07-17 11:07:22,546 : INFO : -6.106 per-word bound, 68.9 perplexity estimate based on a held-out corpus of 2000 documents with 20829 words
2023-07-17 11:07:22,547 : INFO : PROGRESS: pass 1, at document #20000/34993
2023-07-17 11:07:22,703 : INFO : merging changes from 2000 documents into a model of 34993 documents
2023-07-17 11:07:22,705 : INFO : topic #0 (0.333): 0.041*"trump" + 0.017*"pelosi" + 0.011*"election" + 0.009*"people" + 0.008*"democracy" + 0.008*"trumpcoupattempt" + 0

2023-07-17 11:07:24,077 : INFO : topic #1 (0.333): 0.041*"trump" + 0.026*"democracy" + 0.021*"donald" + 0.020*"voted" + 0.020*"mark" + 0.020*"subpoena" + 0.016*"coup" + 0.016*"history" + 0.015*"testify" + 0.015*"new"
2023-07-17 11:07:24,078 : INFO : topic #2 (0.333): 0.142*"trump" + 0.113*"knew" + 0.062*"responsible" + 0.062*"mcconnell" + 0.062*"called" + 0.062*"mitch" + 0.062*"backed" + 0.061*"mccarthy" + 0.061*"kevin" + 0.017*"6th"
2023-07-17 11:07:24,078 : INFO : topic diff=0.149611, rho=0.226476
2023-07-17 11:07:24,084 : INFO : PROGRESS: pass 2, at document #2000/34993
2023-07-17 11:07:24,243 : INFO : merging changes from 2000 documents into a model of 34993 documents
2023-07-17 11:07:24,244 : INFO : topic #0 (0.333): 0.039*"trump" + 0.017*"pelosi" + 0.009*"get" + 0.008*"nancy" + 0.008*"election" + 0.007*"hours" + 0.007*"come" + 0.007*"people" + 0.006*"happened" + 0.006*"hey"
2023-07-17 11:07:24,245 : INFO : topic #1 (0.333): 0.038*"trump" + 0.027*"democracy" + 0.023*"mark" + 0.019

...

2023-07-17 11:07:44,115 : INFO : topic #1 (0.333): 0.041*"trump" + 0.027*"democracy" + 0.022*"donald" + 0.021*"mark" + 0.021*"voted" + 0.020*"subpoena" + 0.017*"coup" + 0.016*"testify" + 0.016*"new" + 0.016*"history"
2023-07-17 11:07:44,116 : INFO : topic #2 (0.333): 0.142*"trump" + 0.115*"knew" + 0.063*"responsible" + 0.062*"mcconnell" + 0.062*"called" + 0.062*"mitch" + 0.062*"backed" + 0.062*"mccarthy" + 0.062*"kevin" + 0.017*"6th"
2023-07-17 11:07:44,116 : INFO : topic diff=0.122534, rho=0.198043
2023-07-17 11:07:44,122 : INFO : PROGRESS: pass 8, at document #2000/34993
2023-07-17 11:07:44,278 : INFO : merging changes from 2000 documents into a model of 34993 documents
2023-07-17 11:07:44,279 : INFO : topic #0 (0.333): 0.039*"trump" + 0.016*"pelosi" + 0.009*"get" + 0.008*"nancy" + 0.008*"election" + 0.007*"hours" + 0.007*"people" + 0.007*"come" + 0.006*"happened" + 0.006*"hey"
2023-07-17 11:07:44,279 : INFO : topic #1 (0.333): 0.038*"trump" + 0.028*"democracy" + 0.024*"mark" + 0.020

2023-07-17 11:07:45,557 : INFO : topic #1 (0.333): 0.033*"trump" + 0.030*"democracy" + 0.028*"mark" + 0.020*"country" + 0.020*"maga" + 0.020*"gop" + 0.019*"violence" + 0.018*"without" + 0.018*"love" + 0.017*"january6thcomm"
2023-07-17 11:07:45,558 : INFO : topic #2 (0.333): 0.138*"trump" + 0.117*"knew" + 0.072*"called" + 0.071*"responsible" + 0.071*"mcconnell" + 0.070*"mccarthy" + 0.070*"mitch" + 0.070*"kevin" + 0.070*"backed" + 0.013*"6th"
2023-07-17 11:07:45,558 : INFO : topic diff=0.156415, rho=0.194270
2023-07-17 11:07:45,854 : INFO : -6.073 per-word bound, 67.3 perplexity estimate based on a held-out corpus of 2000 documents with 20829 words
2023-07-17 11:07:45,855 : INFO : PROGRESS: pass 8, at document #20000/34993
2023-07-17 11:07:46,007 : INFO : merging changes from 2000 documents into a model of 34993 documents
2023-07-17 11:07:46,008 : INFO : topic #0 (0.333): 0.040*"trump" + 0.016*"pelosi" + 0.011*"election" + 0.009*"people" + 0.008*"democracy" + 0.008*"trumpcoupattempt" + 0

2023-07-17 11:07:47,342 : INFO : topic #1 (0.333): 0.041*"trump" + 0.027*"democracy" + 0.022*"donald" + 0.022*"mark" + 0.021*"voted" + 0.020*"subpoena" + 0.017*"coup" + 0.016*"testify" + 0.016*"new" + 0.016*"history"
2023-07-17 11:07:47,342 : INFO : topic #2 (0.333): 0.142*"trump" + 0.115*"knew" + 0.063*"responsible" + 0.062*"mcconnell" + 0.062*"called" + 0.062*"mitch" + 0.062*"backed" + 0.062*"mccarthy" + 0.062*"kevin" + 0.017*"6th"
2023-07-17 11:07:47,342 : INFO : topic diff=0.119976, rho=0.194270
2023-07-17 11:07:47,348 : INFO : PROGRESS: pass 9, at document #2000/34993
2023-07-17 11:07:47,502 : INFO : merging changes from 2000 documents into a model of 34993 documents
2023-07-17 11:07:47,504 : INFO : topic #0 (0.333): 0.039*"trump" + 0.016*"pelosi" + 0.009*"get" + 0.008*"nancy" + 0.008*"election" + 0.007*"hours" + 0.007*"people" + 0.007*"come" + 0.006*"happened" + 0.006*"hey"
2023-07-17 11:07:47,504 : INFO : topic #1 (0.333): 0.038*"trump" + 0.028*"democracy" + 0.024*"mark" + 0.020

2023-07-17 11:07:48,840 : INFO : topic #1 (0.333): 0.033*"trump" + 0.030*"democracy" + 0.028*"mark" + 0.020*"country" + 0.020*"maga" + 0.020*"gop" + 0.019*"violence" + 0.018*"without" + 0.018*"love" + 0.017*"january6thcomm"
2023-07-17 11:07:48,841 : INFO : topic #2 (0.333): 0.138*"trump" + 0.116*"knew" + 0.071*"called" + 0.070*"responsible" + 0.070*"mcconnell" + 0.070*"mccarthy" + 0.070*"mitch" + 0.070*"kevin" + 0.070*"backed" + 0.013*"6th"
2023-07-17 11:07:48,841 : INFO : topic diff=0.153101, rho=0.190705
2023-07-17 11:07:49,135 : INFO : -6.070 per-word bound, 67.2 perplexity estimate based on a held-out corpus of 2000 documents with 20829 words
2023-07-17 11:07:49,136 : INFO : PROGRESS: pass 9, at document #20000/34993
2023-07-17 11:07:49,287 : INFO : merging changes from 2000 documents into a model of 34993 documents
2023-07-17 11:07:49,289 : INFO : topic #0 (0.333): 0.040*"trump" + 0.016*"pelosi" + 0.011*"election" + 0.009*"people" + 0.008*"democracy" + 0.008*"trumpcoupattempt" + 0

2023-07-17 11:07:50,665 : INFO : topic #1 (0.333): 0.041*"trump" + 0.027*"democracy" + 0.022*"mark" + 0.021*"donald" + 0.021*"voted" + 0.020*"subpoena" + 0.017*"coup" + 0.016*"testify" + 0.016*"new" + 0.015*"history"
2023-07-17 11:07:50,665 : INFO : topic #2 (0.333): 0.142*"trump" + 0.115*"knew" + 0.063*"responsible" + 0.062*"mcconnell" + 0.062*"called" + 0.062*"mitch" + 0.062*"backed" + 0.062*"mccarthy" + 0.062*"kevin" + 0.017*"6th"
2023-07-17 11:07:50,666 : INFO : topic diff=0.117458, rho=0.190705
2023-07-17 11:07:50,666 : INFO : LdaModel lifecycle event {'msg': 'trained LdaModel(num_terms=9400, num_topics=3, decay=0.5, chunksize=2000) in 34.39s', 'datetime': '2023-07-17T11:07:50.666876', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 18:29:29) \n[Clang 12.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}


In [14]:
lda2.print_topics()

2023-07-17 11:07:50,671 : INFO : topic #0 (0.333): 0.039*"trump" + 0.014*"pelosi" + 0.009*"get" + 0.008*"hours" + 0.008*"election" + 0.008*"come" + 0.007*"happened" + 0.007*"hey" + 0.007*"oath" + 0.007*"act"
2023-07-17 11:07:50,672 : INFO : topic #1 (0.333): 0.041*"trump" + 0.027*"democracy" + 0.022*"mark" + 0.021*"donald" + 0.021*"voted" + 0.020*"subpoena" + 0.017*"coup" + 0.016*"testify" + 0.016*"new" + 0.015*"history"
2023-07-17 11:07:50,673 : INFO : topic #2 (0.333): 0.142*"trump" + 0.115*"knew" + 0.063*"responsible" + 0.062*"mcconnell" + 0.062*"called" + 0.062*"mitch" + 0.062*"backed" + 0.062*"mccarthy" + 0.062*"kevin" + 0.017*"6th"


[(0,
  '0.039*"trump" + 0.014*"pelosi" + 0.009*"get" + 0.008*"hours" + 0.008*"election" + 0.008*"come" + 0.007*"happened" + 0.007*"hey" + 0.007*"oath" + 0.007*"act"'),
 (1,
  '0.041*"trump" + 0.027*"democracy" + 0.022*"mark" + 0.021*"donald" + 0.021*"voted" + 0.020*"subpoena" + 0.017*"coup" + 0.016*"testify" + 0.016*"new" + 0.015*"history"'),
 (2,
  '0.142*"trump" + 0.115*"knew" + 0.063*"responsible" + 0.062*"mcconnell" + 0.062*"called" + 0.062*"mitch" + 0.062*"backed" + 0.062*"mccarthy" + 0.062*"kevin" + 0.017*"6th"')]